In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import geopandas as gpd
from scipy.stats import norm
from __future__ import print_function, division

In [ ]:
#https://github.com/fedhere/PUI2016_fb55/blob/master/PreMidtermReview.md

# Downloads

In [2]:
#download json y abrir. Ejemplo con plot
import requests
import json

url = 'https://s3.amazonaws.com/sb-public/sbg389_matplotlibrc.json' 
resp = requests.get(url = url)
data = json.loads(resp.text)

plt.rcParams.update(data)


/home/pipe/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/home/pipe/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# OS operations on files

In [27]:
#Download data with wget
os.system("wget https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/mn_mappluto_16v1.zip")

#move to PUIDATA
os.system("mv " + 'mn_mappluto_16v1.zip ' + os.getenv("PUIDATA"))

#change permisions
os.system('chmod -R 777 ' + os.getenv("PUIDATA"))

#Unzip explicit destination folder
os.system('unzip ' + os.getenv("PUIDATA") + "/" + "mn_mappluto_16v1.zip " +\
'-d ' +  os.getenv("PUIDATA"))


0

In [30]:
bsize = gpd.read_file(os.getenv('PUIDATA') + "/MNMapPLUTO.shp")
nrg = pd.read_csv("https://data.cityofnewyork.us/api/views/rgfe-8y2z/rows.csv")
bsize.head(2)

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.000000e+00,None,1592 2 AVENUE,C1-9/TA,None,2,468000.0,1435950.0,1.015450e+09,10885,...,0,1920,None,10028,C1-9,None,None,None,9a,"POLYGON ((997277.2344000041 221816.0936000049,..."
1,1.007230e+09,11/30/2006,263 9 AVENUE,C1-5/R8,None,2,539984.0,11879993.0,1.007238e+09,89203,...,0,1914,None,10001,R8,None,None,None,8d,"POLYGON ((984164.5626000017 211846.0703999996,..."


# BASIC DATA WRANGLING

In [43]:
#RENOMBRAR
nrg.rename(columns={'NYC Borough, Block, and Lot (BBL)':'BBL'}, inplace= True)

#TO NUMERIC, remplaza con NaNS el coerce
nrg['siteEUI'] = pd.to_numeric(nrg['Site EUI(kBtu/ft2)'], errors = 'coerce')
nrg['floorArea']= pd.to_numeric(nrg['Reported Property Floor Area (Building(s)) (ft²)'], errors = 'coerce')

#SELECT
nrg = nrg.copy().loc[:,['BBL','siteEUI','floorArea']]



In [45]:
#DROP NA, axis  0 borra filas, 1 columnas
#how, any con al menos 1, all tiene que ser todas
nrg.dropna(axis=0, how='any', thresh=None, inplace=True)

(16170, 3)
(11457, 3)


In [46]:
#SELECT BROADCASTING
#create a boolean array to use as mask
nrgMaskQ = (bblnrgdata.totalEnergy > 1000) & (bblnrgdata.totalEnergy < bblnrgdata.totalEnergy.quantile(.98))
unitsMaskQ = (bblnrgdata.Units > 1) & (bblnrgdata.Units < bblnrgdata.Units.quantile(.99))
#join both criteria into one
joinMasks = unitsMaskQ & nrgMaskQ

#select
bblnrgdata = bblnrgdata.loc[joinMasks,:].copy()


# MERGE

In [ ]:
#merge data sets by BBL keeping all rows from the energy data set
bblnrgdata = pd.merge(left = nrg, right=bsize, how = 'left', on = 'BBL')

#"how" PARAMETERS:
#how : {‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘inner’
#left: use only keys from left frame (SQL: left outer join)
#right: use only keys from right frame (SQL: right outer join)
#outer: use union of keys from both frames (SQL: full outer join)
#inner: use intersection of keys from both frames (SQL: inner join)
#https://github.com/fedhere/UInotebooks/blob/master/dataWrangling/PandasDataWrangling-Chap7.ipynb

# DISTRIBUTION TABLES

In [47]:
#AREAS BAJO LA CURVA DE LA NOMAL

#Survival function 1 - cdf, but sf is sometimes more accurate). Area de la curva a la derecha del valor 
#sf(x, loc=0, scale=1), 1.65 es el z para 95%, esto me devuelve 5%
norm.sf(1.65, loc = 0, scale = 1,)

0.049471468033648103

In [48]:
#Percent point function (inverse of cdf — percentiles). Me va el valor de Z dada un area bajo la curva. 
#Area total, si quiero a dos colas, divido alpha por 2: 5% / 2 es 0.975 o 1 -  (alpha/2)
#http://images.slideplayer.es/17/5511464/slides/slide_10.jpg
#ppf(q, loc=0, scale=1)
norm.ppf(.95, loc = 0, scale = 1) 


1.6448536269514722

In [46]:
#chi2
from scipy.stats import chi2
#sf(x, df, loc=0, scale=1) me da el % a la derecha de ese valor
print (chi2.sf(x = 3.84, df= 1, loc=0, scale=1))

#ppf(q, df, loc=0, scale=1) me da el valor que me deja a la dercha el 5 % 
print (chi2.ppf(.95, df= 1, loc=0, scale=1))

0.0500435212487
3.84145882069


# Changing columns

In [32]:
#combining with pd.to_numeric, pd.rename

#Map + lambda
delayData['departLagDay'] = delayData.departLag.map(lambda x: x.day)


# REGRESSION

In [ ]:
import statsmodels.formula.api as smf
mod = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df)
res = mod.fit()
print res.summary()
print res.params

#With out intercept
res = smf.ols(formula='Lottery ~ Literacy + Wealth + C(Region) -1 ', data=df).fit()


# PLOTS

In [ ]:
#create a figure with size
fig = plt.figure(figsize=(8,6))

#give axis to that figure and position on the grid
ax = fig.add_subplot(1,1,1)

#generate de plot to the axis
ax.plot(x_variable,y_variable,'bo')

#set labels and title
ax.set_title('Total energy distribution')
ax.set_xlabel('VAR NAME X')
ax.set_Ylabel('VAR NAME y')

#GIVE A CAPTION

In [ ]:
#plotting lines

#BY generating new X and the equations
incomes = np.linspace(0,60000,100)
incomesFitedModel = lm.params[0] + incomes * lm.params[1]

#or

def lmPolFited (x):
    return lmPol.params[0] + x * lmPol.params[1] + (x**2) * lmPol.params[2]

xNew = np.linspace(0,3.5, 100)
y_fit = map(lmPolFited,xNew)
ax.plot(xNew,y_fit,'r-')



#BY passing two points
ax.plot([x0,y0], [x1,y1], 'k--', label = 'equality line')

#range of x from 0 to 3.5
ax.plot([0,3.5],[lmEvU.params[0],lmEvU.params[0] + 3.5 * lmEvU.params[1]],'r')




#HISTOGRAMS for continues variables with probability on the Y axis, give the amount of bins
plt.hist(vector, bins=30, color='red')

#BARPLOT
    ##with pandas y groupby counts (puede ser count_nonzero), y genera un plot con pd.plot
ax = ((df['date'][df['gender'] == 1].groupby([df['date'].dt.weekday]).count()))\
.plot(kind="bar",
      color='SteelBlue',
      label='male')


    ##with bins based on values of the variable 
#create values for cuting the variable in categories
bins = np.arange(15, 99, 5)

#The group by along with the cumsim will get the cumulative count for each of the bins
cs=df.age.groupby(pd.cut(df.age, bins)).agg([count_nonzero]).cumsum()

# TEST

## ChiSquare 

In [36]:
#http://www.psychstat.missouristate.edu/introbook/sbk28m.htm
import numpy as np

def evalChisq(values):
    '''
    Takes the observed table in absoluts, 
    as a array of arrays (one for each row, and the elements are the cols)
    returns a value of the CHI2 statistic
    '''
    values = np.array(values)
    E = np.empty_like(values)
    for j in range(len(values[0])):
        for i in range(2):
            
            E[i][j] = ((values[i,:].sum() * values[:,j].sum()) / 
                        (values).sum())
    return ((values - E)**2 / E).sum()


sample_values_ceojob  = np.array([[0.701 * 564, 0.299 * 564], [0.0305 * 409, 0.965 * 409]])

chisq_ceojob = evalChisq(sample_values_ceojob)


## Goodness of fit

In [ ]:
#https://github.com/fedhere/UInotebooks/blob/master/slides/UI5_PUI2016.pdf

#two sample ks test: two samples come from the same distirbution

#This is a two-sided test for the null hypothesis that 2 independent samples are drawn from the same continuous distribution.
#Parameters: a, b : sequence of 1-D ndarrays
scipy.stats.ks_2samp(data1, data2)
        
#ks test: the sample comes from a model. POWER IN THE CORE

#rvs : str, array or callable MY DATA
#cdf : str or callable DISTRIBUTION TO COMPARE
#If a string, it should be the name of a distribution in scipy.stats. 

scipy.stats.kstest(rvs, cdf, args=(), N=20, alternative='two-sided', mode='approx')[source]
#example
x = np.linspace(-15, 15, 9)
stats.kstest(x, 'norm')




#Anderson-Darling test for data coming from a particular distribution. POWER IN THE TAILS

#the null hypothesis that a sample is drawn from a population that follows a particular distribution.
#x : array_like MY DATA
#dist : {‘norm’,’expon’,’logistic’,’gumbel’,’extreme1’}, optional


scipy.stats.anderson(x, dist='norm')

# LIKELYHOOD

In [ ]:
logLikelihood_Model1 = lmEvU.llf #fewer parameters
logLikelihood_Model2 = lmPol.llf
LR = -2 * (logLikelihood_Model1 - logLikelihood_Model2)

print LR #returns chi square DF are the diference in parameters between the 2 models

#comparison with another model
lmPol.compare_lr_test(lmUvE)


# CREATE VARIABLES IN A LOOP WITH NAME CHANGING

In [1]:
for i in range(2):
    globals()["case_{}".format(i)]=i

case_0

0